# Lab SQL 9
In this lab we will find the customers who were active in consecutive months of May and June. Follow the steps to complete the analysis.


## Create a table `rentals_may` to store the data from rental table with information for the month of May.

In [58]:
import pymysql
from sqlalchemy import create_engine
import pandas as pd
import getpass  # to get the password without showing the input / password of sql workbench -- which is for most of ppl 'admin'
password = getpass.getpass()

········


In [59]:
connection_string = 'mysql+pymysql://root:' + password + '@localhost/sakila'
engine = create_engine(connection_string)

In [4]:
engine.execute("CREATE TABLE rentals_may LIKE rental;")

## Insert values in the table `rentals_may` using the table rental, filtering values only for the month of May.
 

In [5]:
engine.execute('''
               INSERT INTO rentals_may
               SELECT * FROM rental
               WHERE MONTH(rental_date) = 5;
               ''')

## Create a table `rentals_june` to store the data from rental table with information for the month of June.


In [10]:
engine.execute("CREATE TABLE rentals_june LIKE rental;")

## Insert values in the table `rentals_june` using the table rental, filtering values only for the month of June.
 

In [11]:
engine.execute('''
               INSERT INTO rentals_june
               SELECT * FROM rental
               WHERE MONTH(rental_date) = 6;
               ''')

## Check the number of rentals for each customer for May.


In [ ]:
# Operation on SQL side

# INSERT INTO rentals_may
# SELECT * FROM rental
# WHERE MONTH(rental_date) = 5;

## Check the number of rentals for each customer for June.
 

In [ ]:
# Operation on SQL side

# INSERT INTO rentals_june
# SELECT * FROM rental
# WHERE MONTH(rental_date) = 6;

## Create a Python connection with SQL database and retrieve the results of the last two queries (also mentioned below) as dataframes:

  - Check the number of rentals for each customer for May
  - Check the number of rentals for each customer for June

    **Hint**: You can store the results from the two queries in two separate dataframes.
 

In [13]:
data_may = pd.read_sql_query('SELECT * FROM rentals_may', engine)
data_may.shape

(1156, 7)

In [14]:
data_may.columns

Index(['rental_id', 'rental_date', 'inventory_id', 'customer_id',
       'return_date', 'staff_id', 'last_update'],
      dtype='object')

In [15]:
# just for practice getting data by execute operation 
result = engine.execute('SELECT * FROM rentals_june') 
rows = [row for row in result]
data_june = pd.DataFrame(rows)
data_june.columns = data_may.columns

In [21]:
data_june.head()

,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update
0,1158,2005-06-14 22:53:33,1632,416,2005-06-18 21:37:33,2,2006-02-15 21:30:53
1,1159,2005-06-14 22:55:13,4395,516,2005-06-17 02:11:13,1,2006-02-15 21:30:53
2,1160,2005-06-14 23:00:34,2795,239,2005-06-18 01:58:34,2,2006-02-15 21:30:53
3,1161,2005-06-14 23:07:08,1690,285,2005-06-21 17:12:08,1,2006-02-15 21:30:53
4,1162,2005-06-14 23:09:38,987,310,2005-06-23 22:00:38,1,2006-02-15 21:30:53


In [18]:
data_june.dtypes

rental_id                int64
rental_date     datetime64[ns]
inventory_id             int64
customer_id              int64
return_date     datetime64[ns]
staff_id                 int64
last_update     datetime64[ns]
dtype: object

## Write a function that checks if customer borrowed more or less films in the month of June as compared to May.
 
 **Hint**: For this part, you can create a join between the two dataframes created before, using the merge function available for pandas dataframes. Here is a link to the documentation for the [merge function](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html).

In [64]:
query1 = '''
         SELECT customer_id, count(rental_id) as number_of_rentals
         from rentals_may 
         group by customer_id
         '''

In [65]:
query2 = '''
         SELECT customer_id, count(rental_id) as number_of_rentals
         from rentals_june
         group by customer_id
         '''

In [66]:
data_may2 = pd.read_sql_query(query1, engine)
data_may2

,customer_id,number_of_rentals
0,1,2
1,2,1
2,3,2
3,5,3
4,6,3
...,...,...
515,594,4
516,595,1
517,596,6
518,597,2


In [67]:
data_june2 = pd.read_sql_query(query2, engine)
data_june2

,customer_id,number_of_rentals
0,1,7
1,2,1
2,3,4
3,4,6
4,5,5
...,...,...
585,595,2
586,596,2
587,597,3
588,598,1


In [101]:
merged_data = pd.merge(data_may2, data_june2, how='outer', on='customer_id')

In [108]:
merged_data.columns = ['customer_id','may','june']
merged_data =  merged_data.fillna(0)
merged_data

,customer_id,may,june
0,1,2.0,7.0
1,2,1.0,1.0
2,3,2.0,4.0
3,5,3.0,5.0
4,6,3.0,4.0
...,...,...,...
593,583,0.0,6.0
594,585,0.0,4.0
595,591,0.0,3.0
596,592,0.0,5.0


In [114]:
def rental_checker(customer_id):
    temp_data = merged_data[merged_data['customer_id'] == customer_id]
    if len(temp_data) == 0:
        return print('There were no rental in May nor June.')
    elif sum(temp_data['may']) > sum(temp_data['june']):
        return print('More rentals in May.')
    elif sum(temp_data['may']) < sum(temp_data['june']):
        return print('More rentals in June.')
    elif sum(temp_data['may']) == sum(temp_data['june']):
        return print('Rentals in May and June were same.')

In [116]:
rental_checker(2)

Rentals in May and June were same.


In [ ]:
# updated function by using Abhi's code